In [111]:
import requests
import json
from bs4 import BeautifulSoup
from typing import List
import time
from multiprocessing import Pool
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def extract_urls_lists(search_page_url: str) -> List[str]:
    response = requests.get(search_page_url, params = {})
    soup = BeautifulSoup(response.text, 'html.parser')

    list_of_urls=[]

    for article in soup.find_all('iw-search-card-rendered'):
        for link in article.find_all('a'):
            list_of_urls.append(link.get('href'))
    return list_of_urls

def extract_x_urls(number_of_urls: int=30) -> List[str]:
    immo_base_url = "https://www.immoweb.be/fr/recherche/maison/a-vendre?countries=BE&page="
    immo_end_url = "&orderBy=relevance"
    searchpages_urls_list =[]

    for index in range(1,round(min(334,number_of_urls/30 + 1))):
        full_url = immo_base_url + str(index) + immo_end_url
        searchpages_urls_list.append(full_url)
    
    full_urls_list =[]

    with Pool() as pool:
        full_urls_list=list(pool.map(extract_urls_lists, searchpages_urls_list))    
    return sum(full_urls_list,[])


In [112]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
pd.set_option('display.width', None)
def make_one_data_frame(url):
  session = requests.Session()
  retry = Retry(connect=3, backoff_factor=0.5)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  session.mount('https://', adapter)
  response = session.get(url)
  test_page = response.text

  dfs = pd.read_html(test_page)

  full_df = pd.concat(dfs).replace("", None).dropna(thresh=2).T
  full_df.columns = full_df.iloc[0]
  full_df = full_df[1:]
  return full_df
  # pd.read_html reads in all tables and returns a list of DataFrames



In [114]:

list_of_urls = extract_x_urls(90)
print('list ready')
with Pool() as pool:
        dataframes_list=list(pool.map(make_one_data_frame, list_of_urls))    

full_df= pd.concat(dataframes_list, )

full_df

list ready


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [121]:
pd.concat(dataframes_list, )

0                   À partir de              Disponible le   
1  Après la signature de l’acte  30 septembre 2023 - 00:00  \

0 Quartier ou lieu-dit Année de construction État du bâtiment   
1       Liège environs                  1850              Bon  \

0 Largeur de la façade à rue Nombre de façades Type d’environnement   
1                        7 m                 3                Isolé  \

0  Visite virtuelle      Surface habitable      Surface du salon   
1  Visite virtuelle  230  m² mètres carrés  12  m² mètres carrés  \

0 Salle à manger     Type de cuisine Surface de la cuisine Chambres   
1            Oui  Américaine équipée  11  m² mètres carrés        5  \

0 Surface de la chambre 1 Surface de la chambre 2 Surface de la chambre 3   
1    15  m² mètres carrés    15  m² mètres carrés    26  m² mètres carrés  \

0 Surface de la chambre 4 Surface de la chambre 5 Salles de bains   
1    12  m² mètres carrés    13  m² mètres carrés               2  \

0 Salles de douche Toilettes Bureau Surface de l'espace professionnel Cave   
1                2         2    Oui              73  m² mètres carrés  Oui  \

0 Grenier Meublé      Surface du terrain Largeur du terrain à rue   
1     Oui    Non  432  m²  mètres carrés              7 m  mètres  \

0 Raccordement à l'égout       Surface du jardin Ascenseur   
1               Connecté  339  m²  mètres carrés       Non  \

0 Accessible aux personnes avec un handicap Câble TV Piscine   
1                                       Non      Oui     Non  \

0               Consommation d'énergie primaire Classe énergétique   
1  403  kWh/m²  kilowattheure par mètres carrés                  E  \

0 Numéro du rapport PEB   Emission CO₂   
1        20220908028590  101 kg CO₂/m²  \

0 Consommation théorique totale d'énergie primaire Type de chauffage   
1                                     28418 kWh/an            Mazout  \

0 Double vitrage Surface constructible totale au sol   
1            Oui               93  m²  mètres carrés  \

0 Autorisation de lotissement Droit d'achat prioritaire possible   
1                         Non                                Non  \

0 Procédure pour violation de la réglementation urbanistique   
1                                                        Non  \

0 Type de zone inondable   
1     Zone non inondable  \

0 Affectation urbanistique la plus récente sur la base des dénominations utilisées dans le registre des plans   
1                                                               Zone d'habitat (résidentiel, urbain ou rural)  \

0                                                                   Prix   
1  Faire offre à partir de  235.000 €  Faire offre à partir de  235000 €  \

0 Revenu cadastral Immeuble de rapport                               Adresse   
1     593 €  593 €                 Oui  Rue de la Wallonie 2A 4680  - Oupeye  \

0             Site internet Référence externe                   À partir de   
1  http://www.nigel-immo.be           4934898  Après la signature de l’acte  \

0              Disponible le Quartier ou lieu-dit Année de construction   
1  30 septembre 2023 - 00:00       Liège environs                  1850  \

0 État du bâtiment Largeur de la façade à rue Nombre de façades   
1              Bon                        7 m                 3  \

0 Type d’environnement  Visite virtuelle      Surface habitable   
1                Isolé  Visite virtuelle  230  m² mètres carrés  \

0      Surface du salon Salle à manger Type de cuisine Surface de la cuisine   
1  12  m² mètres carrés            Oui     Pas équipée  11  m² mètres carrés  \

0 Chambres Surface de la chambre 1 Surface de la chambre 2   
1        5    15  m² mètres carrés    15  m² mètres carrés  \

0 Surface de la chambre 3 Surface de la chambre 4 Surface de la chambre 5   
1    26  m² mètres carrés    12  m² mètres carrés    13  m² mètres carrés  \

0 Salles de bains Salles de douche Toilettes     Surface de bureau Bureau   
1               2              